In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor 
from pycor import utils, korlang

stopwatch = utils.StopWatch()


docsize = 20
model_size = 3000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

pycor.loadmodel(model_path)

parser = korlang._trainer
keywordutils = korlang._keywordutils

Init Trainer
Init DocResolver
Loading Model from ../../output/3000/model/
Load  ../../output/3000/model//model.csv   소요시간: 6.036
Load  ../../output/3000/model//tails.csv   소요시간: 0.015


In [2]:

def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")


3667 files


In [3]:

def printKwywords(keywords, words_array):
    print("===========")
    print(keywords)

def summarizeDocument(file, rate=0.1, count=3):
    if file.endswith(".txt") :
        print(file)
        sentence_array = parser.loadfile(file)
        keywords = keywordutils.extractKeywords(sentence_array, rate)
        if len(keywords) < 1:
            rate /= 2
            keywords = keywordutils.extractKeywords(sentence_array, rate)
        print("===========")
        print(keywords)
        print("------------")
        sentences = keywordutils.abstractDocument(keywords, sentence_array, count)
        korlang.printSentences(sentences)
        return sentence_array


def summarizeDocument2(file, keywords, count=3):
    if file.endswith(".txt") :
        print(file)
        sentence_array = parser.loadfile(file)
        print("===========")
        print(keywords)
        print("------------")
        sentences = keywordutils.abstractDocument(keywords, sentence_array, count)
        dr.printSentences(sentences)
        return sentence_array


def extractKw(file, rate=0.1):
    if file.endswith(".txt") :
        print(file)
        sentence_array = parser.loadfile(file)
        keywords = keywordutils.extractKeywords(sentence_array,rate)
        printKwywords(keywords, sentence_array)


stopwatch.start()

for file in files[:docsize]:
#     extractKw(file, 0.05)
    summarizeDocument(file, 0.05, 3)

# extractKw("../data/NP/mp-all.txt",0.1)
# summarizeDocument("../data/NP/mp-all.txt",0.1)
# summarizeDocument("../data/wiki/%EB%8B%A8%EB%B0%B1%EC%A7%88.txt",0.1)
# summarizeDocument("samples/01_sampletext.txt")
# summarizeDocument("../data/wiki/%EC%A0%95%EB%B3%B4%20%EC%9D%B4%EB%A1%A0.txt")

print("Loading Texts: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

../samples/news/new1_joins.txt
[countmap] {HTTPS: 1, JOINS: 1, COMARTICLE22903743: 1, CLOCRSSNEWSTOTAL_LIST: 1, 솔릭: 5, 북상광주전남: 1, 지역: 6, 태풍예비특보: 4, 제19호: 1, 태풍: 6, SOULIK: 1, 북상함: 1, 광주: 4, 전남: 3, 광주기상청: 3, 오: 2, 오후: 5, 22개: 1, 시군: 1, 지역흑산도홍도: 1, 남해서부앞바다서해남부먼바다서해남부앞바다: 1, 새벽: 1, 여수순천광양보성고흥강진해남장흥완도거문도초도: 1, 강풍예비특보: 2, 남해서부동쪽먼바다: 2, 남해서부앞바다: 1, 풍랑예비특보: 2, 오전: 2, 시각: 1, 흑산도홍도: 1, 서해남부먼바다서해남부앞바다: 1, 이날: 1, 3시: 2, 가고시: 1, 380: 1, 부근: 3, 해상: 4, 시속: 2, 23: 1, 속도: 1, 북서진하: 1, 중심기압: 1, 950HPA: 1, 최대풍속: 1, 초속: 1, 43M: 1, 155: 1, 중형급: 1, 서귀포: 1, 9시: 1, 목포: 1, 남서쪽: 1, 120: 1, 직간접적: 1, 밤: 3, 100250: 1, 내리: 2, 지리산: 1, 산간: 1, 400: 1, 폭우: 1, 곳: 3, 바람: 2, 영향권: 2, 드: 1, 해안: 2, 산지: 1, 최대순간풍속: 2, 이상: 1, 매우: 2, 부: 1, 내륙: 1, 2030: 1, 안팎: 1, 남해서해안: 1, 너울: 1, 높: 1, 물결: 1, 파도: 1, 방파제: 1, 해안도로: 1, 관계자: 1, 들면서: 1, 강풍풍랑: 1, 인명: 1, 재산피해: 1, 저지대: 1, 침수피해: 1, 강풍: 1, 시설물: 1, 안전사고: 1, 유의해달: 1}
[selected] {}
[countmap] {HTTPS: 1, JOINS: 1, COMARTICLE22903743: 1, CLOCRSSNEWSTOTAL_LIST: 1, 솔릭: 5, 북상광주전남: 1, 지역: 6, 태풍예비특보

TypeError: sequence item 16: expected str instance, NoneType found

In [ ]:
words_array_list = []

for file in files[:4]:
    words_array = summarizeDocument(file, 0.05, 3)
    words_array_list.append(words_array)

print("======== Merge News ==========")
sentences = keywordutils.mergeDocuments(words_array_list, 0.3, 6)

korlang.printSentences(sentences)


In [ ]:
words_array_list = []

for file in files[27:33]:
    words_array = summarizeDocument(file, 0.05, 3)
    words_array_list.append(words_array)

print("======== Merge News ==========")
sentences = keywordutils.mergeDocuments(words_array_list, 0.1, 6)

korlang.printSentences(sentences)